# LLM Demo

- Quickstart with OpenAI API
- Show the prompting powers.

In [4]:
from openai import OpenAI
import os

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the wonders of NixOS."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the realm of computing's fierce domain,\nWhere myriad systems seek to proudly reign,\nThere lies a wonder, unique and free,\nA majestic entity, called NixOS, you see.\n\nAmidst the chaos of battles waged in code,\nNixOS stands tall, admirable ode,\nFor it embraces a spirit, quite sublime,\nA masterpiece of harmony, that defies time.\n\nLike a symphony conductor, it orchestrates,\nA myriad of packages, at varied rates,\nWith its declarative nature, it weaves,\nA tapestry of stability, where joy cleaves.\n\nIn the heart of NixOS, the mighty Nix resides,\nA package manager, a constant guide,\nWith purity it reigns, unwavering grace,\nBringing reproducibility to a world's fast pace.\n\nConfiguration, a breeze like gentle wind,\nWith its declarative language, do we find,\nA precise expression of desires untold,\nStructured and clear, a sight to behold.\n\nRollbacks become effortless, without strife,\nWith its atomic updates and safeguards rife,\nOh, the won

In [25]:
!poetry add tqdm
!poetry add numpy
!poetry add pandas

The following packages are already present in the pyproject.toml and will be skipped:

  • tqdm

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
The following packages are already present in the pyproject.toml and will be skipped:

  • numpy

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
Using version ^2.2.0 for pandas

Updating dependencies
Resolving dependencies... (1.2s)Resolving dependencies... (0.1s)

Package operations: 3 installs, 0 updates, 0 removals

  • Installing pytz (2024.1): Pending...
  • Installing tzdata (2024.1): Pending...
  • Installing tzdata (2024.1): Pending...
  • Installing pytz (2024.1): Installing...
  • Installing tzdata (2024.

# Initial Observations

- Wow, it's really creative! How can we do more with this?
- Let's do some sentiment classification

In [12]:
writing_like_me = [
    {"role": "system", "content": "You classify sentiment"},
    {"role": "user", "content": "I hate this restaurant, it sucks."}
]

def get_completions(messages, model='gpt-3.5-turbo'):
    return client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        temperature=0
    ).choices[0].message.content
    
get_completions(writing_like_me)

'Sentiment: Negative'

# In-Context Learning

- The above example is showing zero-shot learning
- Give the model a task, with no examples.
- Do we like this format, can we improve the format?

In [13]:
writing_like_me += [
    {"role": "assistant", "content": "{'sentiment': 'negative'}"},
    {"role": "user", "content": "I love this place!"}
]

get_completions(writing_like_me)

"{'sentiment': 'positive'}"

# One-Shot Learning

- Given a single example, GPT-3.5 figured out what format we want our data in and used that for the next response.
- Giving examples as part of a prompt is 1-N shot learning, as opposed to training data being required

In [15]:
import tqdm

with open("/home/jacob/org/roam/20240112101043-todos.org", "r") as f:
    data = f.readlines()
    todos = [t for t in data if t.startswith('* TODO')]

# Generate embeddings for all my todo tasks.
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

embeddings = [get_embedding(x) for x in tqdm.tqdm(todos)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:12<00:00,  4.32it/s]


# Embeddings Based Search

- Embeddings are the number-based representation of text.
- Embedding models are trained to help learn relationships between tokens.
- Can use generated embeddings to also build search applications.
- Here's an example with my todo list.

In [18]:
# import required libraries
import numpy as np
from numpy.linalg import norm

question = "Run whole system today with new parameters"
todos_with_embedding = list(zip(embeddings, todos))

# compute cosine similarity
def cosine_similarity(e1, e2):
    return np.dot(e1, e2) / (norm(e1) * norm(e2))

def get_cosine_similarity(text):
    embedding = get_embedding(text)
    sims = [(cosine_similarity(embedding, e[0]), e[0], e[1]) for e in todos_with_embedding]
    return sorted(sims, key=lambda x: x[0])[0][2]

get_cosine_similarity(question)

'* TODO [#B] - Secure connection to SignalR Hub/Client :vasi:driver:\n'

# Quick and Dirty Search Engine

- Uses OpenAI embeddings generated in previous step.
- Generate embedding for input text.
- Calculate cosine similarity for each of the other embeddings we have.
- Return the text that is the most similar.

## Extending with RAG (Retrieval Augmented Generation)

- Use the retrieved `n` prompts as context for a new prompt

In [19]:
import pandas as pd

data = pd.read_csv('../datasets/topical_chat.csv')
first_conversation_messages = data.query('conversation_id == 1').message

/tmp/ipykernel_402346/3824448645.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Summarizing Discussion Content

- Given a Chatbot conversation, can we summarize it?
- Rolling summary to reduce the prompt length, or to handle conversations longer than the context window?
- Let's generate a rolling summary, compressing information.

In [20]:
def get_summary_prompt(summary, message):
    print(f"summary: {summary}\n")
    print(f"message: {message}\n")
    return [
        {'role': 'system', 'content': 'You exist to summarize content, compress the information as much as possible without losing meaning'},
        {'role': 'user', 'content': f'Summarize this existing summary and the following message \n{summary} \n{message}'}
    ]


summary = ''
for message in first_conversation_messages:
    summary = get_completions(get_summary_prompt(summary, message))
    print(f"completion: {summary}")
    print("="*100)


summary: 

message:  Are you a fan of Google or Microsoft?

completion: Summary: The question asks if the person prefers Google or Microsoft.

Message: The message is not provided.
summary: Summary: The question asks if the person prefers Google or Microsoft.

Message: The message is not provided.

message:  Both are excellent technology they are helpful in many ways. For the security purpose both are super.

completion: The question asks about preference between Google and Microsoft. Both are excellent technology and helpful in many ways, especially for security purposes.
summary: The question asks about preference between Google and Microsoft. Both are excellent technology and helpful in many ways, especially for security purposes.

message:  I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense. 

completion: The question is about preference between Google and Microsoft, both excellent technology options with many benefits, pa

# Results

- Watch as the summary is updated in real-time
- This is a particularly silly example

In [21]:
def get_classification_prompt(text, *args):
    return [
        {'role': 'user', 'content': f'Classify the following text: {text}'},
        *args
    ]

get_completions(get_classification_prompt('This is a silly thing you are asking me to do'))

'Sentiment: Negative'

# Combine In-Context Learning and Prompt Engineering

- We asked the LLM to classify text without any context.
- Sentiment was used
- What if we want emotions

In [22]:
get_completions(
    get_classification_prompt(
        'This is a silly thing you are asking me to do', 
        {'role': 'assistant', 'content': 'Angry, but comical'}, 
        {'role': 'user', 'content': 'What an ass-backwards way to screw in a lightbulb'}
    ))

'Sarcastic, critical'

# What does this show?

One-shot learning

## What if we don't want to provide more examples? Reduce the prompt length?

- Use prompt engineering

In [23]:
# Using Prompt Engineering
def get_classification_prompt_with_context(text, *args):
    return [
        {'role': 'user', 'content': f'Using the categories happy, mad, or sad, classify the following text: {text}'},
        *args
    ]

get_completions(get_classification_prompt_with_context(
    "This is a silly thing you are asking me to do", 
    {'role': 'assistant', 'content': 'mad'}, 
    {'role': 'user', 'content': 'You are a terrible human being and it makes me want to cry'}))

'sad'

In [75]:
get_completions(get_classification_prompt_with_context(
    "This is a silly thing you are asking me to do", 
    {'role': 'assistant', 'content': 'mad'}, 
    {'role': 'user', 'content': 'You are a terrible human being and it makes me want to cry'},
    {'role': 'assistant', 'content': 'sad'},
    {'role': 'user', 'content': 'I am so upset right now, no one could calm me down'}))

'mad'